In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end

#Pkg.build("PyCall")
using JuliaProbo, Distributed
using Seaborn: heatmap, savefig
gr()

In [ ]:
mutable struct PolicyEvaluator
    pose_min::Vector{Float64}
    pose_max::Vector{Float64}
    reso::Vector{Float64}
    goal::Goal
    index_nums::Vector{Int64}
    value_function_::AbstractArray{Float64, 3}
    final_state_flags_::AbstractArray{Float64, 3}
end

function set_final_state(
        reso::Vector{Float64},
        goal::Goal,
        pose_min::Vector{Float64},
        index::Tuple{Int64, Int64, Int64}
    )
    x_min, y_min, θ_min = pose_min .+ reso .* ([index...] .- 1)
    x_max, y_max, θ_max = pose_min .+ reso .* ([index...])
    
    corners = [[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]]
    return convert(Float64, all( [inside(goal, corner) for corner in corners] ))
end

# constructor
function PolicyEvaluator(reso::Vector{Float64}, goal::Goal, lowerleft=[-4.0, -4.0], upperright=[4.0, 4.0])
    pose_min = vcat(lowerleft, [0.0])
    pose_max = vcat(upperright, [2pi])
    
    index_nums = [convert(Int64, round( (pose_max[i] - pose_min[i])/reso[i] )) for i in 1:3]
    v = zeros(Float64, index_nums[1], index_nums[2], index_nums[3])
    f = zeros(Float64, index_nums[1], index_nums[2], index_nums[3])
    
    @distributed for id1 in 1:index_nums[1]
        @distributed for id2 in 1:index_nums[2]
            @distributed for id3 in 1:index_nums[3]
                index = (id1, id2, id3,)
                val = set_final_state(reso, goal, pose_min, index)
                @inbounds f[index...] = val
                @inbounds v[index...] = (val == 1.0) ? goal.value : (-100.0)
            end
        end
    end
    
    return PolicyEvaluator(pose_min, pose_max, reso, goal, index_nums, v, f)
end

In [ ]:
pe = PolicyEvaluator([0.2, 0.2, pi/18], Goal(-3.0, -3.0));

In [ ]:
println(typeof(pe.value_function_))
println(size(pe.value_function_))
v = pe.value_function_
p = heatmap(rotl90(v[:, :, 1]))
savefig("images/ch10_policy_evaluation2.png")

<img src="images/ch10_policy_evaluation2.png">